In [103]:
import pandas as pd
import numpy as np
import yfinance as yf


### Parameters
`mu` value it reverts to over time

`theta` determines how fast it reverts to mu. When `theta=1` means 63% of deviation dissapears in 1 time unit
* this means if `theta = 0.1` it takes 10 time units to revert back 63% of mu

`dW_t` determines when random noise enter the system 

`vol` is how large the spikes are in scale 

`var` variance of shock per unit time
`

In [104]:
import yfinance as yf
import pandas as pd

# Download all historical data for AAPL
data = yf.download("CAD=X", period="1mo")

# Keep only the Close column and rename it
df = data[["Close"]].rename(columns={"Close": "CAD"}).reset_index()

# Ensure no multi-level columns remain
df.columns = ["Date", "AAPL"]

df.set_index('Date', inplace=True)



/var/folders/yd/5_1tn8r14mbd7pb95n_8jby80000gn/T/ipykernel_22343/1354539034.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("CAD=X", period="1mo")
[*********************100%***********************]  1 of 1 completed


In [105]:
## long term mean level
def get_miu(df:pd.DataFrame)  -> float:
    return df.mean().iloc[0]

In [106]:
def get_Xt(df:pd.DataFrame)-> float:
    return df.iloc[-1,0]

In [107]:
def get_dt (df:pd.DataFrame) -> float:
    return 1

In [108]:
def get_theta(df:pd.DataFrame) -> float:
    ## time = 1, due to daily returns
    time = get_dt(df)
    rho = df.iloc[:,0].autocorr(lag=1)
    return -(1/time) * np.log(rho) 


In [109]:
def get_var(df:pd.DataFrame, theta:float, dt: float, mu:float) -> float:
    df = df.iloc[:,0].values
    exp1 = np.exp(-theta * dt)
    exp2 = np.exp(-2 * theta * dt)

    diff = df[1:] - df[:-1] * exp1 - mu * (1 - exp1)
    numerator = 2 * theta * np.sum(diff**2)
    denominator = len(df) * (1 - exp2)

    sigma2 = numerator / denominator
    return sigma2

In [110]:
def get_dXt(theta:float, mu: float, Xt:float, dt: float, var: float, dWt: float) -> float:
    return (theta * (mu - Xt)* (1/dt)) +( np.sqrt(var) * dWt)
 

In [111]:
mu = get_miu(df)
time = get_dt(df)
theta = get_theta(df)
X_t = get_Xt(df)
var = get_var(df, theta, time, mu)
dX_t = get_dXt(theta, mu, X_t, time, var, np.sqrt(time))

In [112]:
# import sys
# print(sys.executable, sys.version)  # sanity check: which Python is running?

# # install yfinance into THIS exact interpreter
# import subprocess, sys as _sys
# subprocess.check_call([_sys.executable, "-m", "pip", "install", "--upgrade", "pip", "yfinance", "pandas"])


In [113]:
mu

1.4003382765728494

In [114]:
theta

0.29070151882421075

In [115]:
var

1.034781374618161e-05

In [116]:
np.sqrt(var)*np.sqrt(252)

0.051065145295375064

In [117]:
np.sqrt(0.005)

0.07071067811865475